### Web Scrapping Exercise
WQD180102 Ng Wei Xin
WQD180104 Tan Bing Shien

In [1]:
# pip install requests
# pip install beautifulsoup4

import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

# using requests to fetch pages
# Note: Python Selenium library can be used for javascript pages (by imitating browser behaviour)

# bs4: Html-Parser (or XML), to navigate through html documents

In [ ]:
# get url content using python-requests
page = requests.get('https://www.thestar.com.my/news/latest')
contents = page.content

# parse html document in bs4
soup = BeautifulSoup(contents, 'html.parser')
# print(soup.prettify())

# Create empty dict
# Note: using news-id as identifier -> prevent duplication
myList = {}

# use class "timeline-content" to identify news section
for tag in soup.find_all(class_= "timeline-content", limit=15):

    # further navigate to "h2" tag
    for subtag in tag.find('h2'):
        
        # get String (news headline), trim / strip linebreaks
        strNews = "".join(line.strip() for line in subtag.string.split("\n"))
        idNews = "".join(line.strip() for line in subtag['data-content-id'].split("\n"))
        
        # crawl sub page, get Date / Timestamp
        newsHref = subtag['href']
        subPage = requests.get(newsHref)
        subContent = subPage.content
        subSoup = BeautifulSoup(subContent, 'html.parser')
        timeTag = "".join(line.strip() for line in subSoup.find('time', {'class': 'timestamp'}).string.split("\n"))
        dateTag = "".join(line.strip() for line in subSoup.find('p', {'class': 'date'}).string.split("\n"))
        
        # (print) News-ID: News-String
        print("News-ID ("+idNews+"):")
        print(strNews)
        print(newsHref)
        print(dateTag)
        print(timeTag)
        print()
        
        # Add entry to list 
        myList[subtag['data-content-id']]=strNews

In [3]:
def recrawl(objDF):
    page = requests.get('https://www.thestar.com.my/news/latest')
    contents = page.content

    soup = BeautifulSoup(contents, 'html.parser')

    for tag in soup.find_all(class_= "timeline-content", limit = 15):
        for subtag in tag.find('h2'):
            
            # News-ID
            idNews = "".join(line.strip() for line in subtag['data-content-id'].split("\n"))
                        
            # Check for duplication, proceed only for new item
            # else skip to save computation
            if not df['News_ID'].str.contains(idNews).any():
                
                # Headline
                strNews = "".join(line.strip() for line in subtag.string.split("\n"))
                
                # Category
                catNews = "".join(line.strip() for line in subtag['data-content-category'].split("\n"))
                
                # Timestamp (spider on respective news page)
                newsHref = subtag['href']
                subPage = requests.get(newsHref)
                subContent = subPage.content
                subSoup = BeautifulSoup(subContent, 'html.parser')
                dateNews = "".join(line.strip() for line in subSoup.find(class_="date").string.split("\n"))
                timeNews = "".join(line.strip() for line in subSoup.find(class_="timestamp").string.split("\n"))
                
                # Add info to list
                objDF = objDF.append(pd.Series([idNews, dateNews, timeNews, catNews, strNews, newsHref], index=objDF.columns), ignore_index=True)

    return objDF


In [ ]:
# Create Empty DataFrame:
df= pd.DataFrame(columns=["News_ID", "Date", "Timestamp", "Category", "Headline", "URL"])

# Endless-Loop
# while True: 
    
i = 9 # Test: using 9 counts (1.5 hours)


# while i>0:
while True:
    
    df = recrawl(df)
    print(df)
    
    # Export dataframe to csv file
    # df.to_csv(r'C:\Users\FORGE-15 I7\OneDrive - AsiaPay Limited\Sem 3\WQD7005 DATA MINING\dataset.csv', index = False)
    df.to_csv(r'C:\Users\ngwei\Desktop\Crawling\dataset.csv', index = False)
    
    i=i-1
    time.sleep(600) # Re-Crawl every 10 minutes


   News_ID                    Date    Timestamp        Category  \
0   229453  Wednesday, 11 Mar 2020  9:13 PM MYT      News/World   
1   229452  Wednesday, 11 Mar 2020  9:11 PM MYT      News/World   
2   229451  Wednesday, 11 Mar 2020  9:09 PM MYT      News/World   
3   229450  Wednesday, 11 Mar 2020  9:08 PM MYT      News/World   
4   229446  Wednesday, 11 Mar 2020  8:50 PM MYT     News/Nation   
5   229444  Wednesday, 11 Mar 2020  8:50 PM MYT   Sport/Cricket   
6   229442  Wednesday, 11 Mar 2020  8:47 PM MYT      News/World   
7   229440  Wednesday, 11 Mar 2020  8:41 PM MYT      News/World   
8   229439  Wednesday, 11 Mar 2020  8:40 PM MYT      News/World   
9   229438  Wednesday, 11 Mar 2020  8:34 PM MYT  Sport/Football   
10  229435  Wednesday, 11 Mar 2020  8:30 PM MYT      News/World   
11  229436  Wednesday, 11 Mar 2020  8:30 PM MYT   News/Regional   
12  229434  Wednesday, 11 Mar 2020  8:28 PM MYT     News/Nation   
13  229432  Wednesday, 11 Mar 2020  8:24 PM MYT      News/Worl

In [ ]:
# save to csv
# df.to_csv(r'C:\Users\ngwei\Desktop\dataset.csv', index = False)


# # Save scraped content into a txt file

# file1 = open("tmpcrawl.txt","w") 

# # file1.write("Hello \n") 
# # file1.write(str(soup.prettify())) 
# file1.write(str(myList)) 

# file1.close() 


In [ ]:
df